# 8-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.  
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. `MovieLens 1M Dataset` 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

`MovieLens 1M Dataset` 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip
```

### 1) 데이터 준비와 전처리

---

Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import os
import pandas as pd

rating_file_path = os.getenv(
    'HOME') + '/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::',
                      names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 이번 프로젝트에선 timestamp를 사용하지 않으므로 제거
ratings = ratings.drop("timestamp", axis=1)
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating'] >= 3]
filtered_data_size = len(ratings)

print(
    f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(
    f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating': 'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path = os.getenv('HOME') + '/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

### 2) 분석해 봅시다.

---

- ratings에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

- rating에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

- 가장 인기 있는 영화 30개(인기순)

In [9]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

id로만 출력돼 무슨 영화인지 알 수가 없다.

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

---

In [10]:
# movie_id의 끝부분을 확인해 추가할 id를 정한다.
movies.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [11]:
# movies에 추가할 내용
my_favorites = [
    {'movie_id': [3953],
     'title': 'About Time (2013)',
     'genre': 'Comedy, Drama, Fantasy'},
    {'movie_id': [3954],
     'title': 'If Only (2004)',
     'genre': 'Comedy, Drama, Fantasy'},
    {'movie_id': [3955],
     'title': 'Avengers: Endgame (2019)',
     'genre': 'Action, Adventure, Drama'},
    {'movie_id': [3956],
     'title': 'Spider-Man: Far From Home (2019)',
     'genre': 'Action, Adventure, Sci-Fi'},
    {'movie_id': [3957],
     'title': 'Iron Man Three (2013)',
     'genre': 'Action, Adventure, Sci-Fi'}
]

# 중복된 값이 없으면 추가
for my_favorite in my_favorites:
    if not movies.isin(my_favorite['movie_id'])['movie_id'].any():
        movies = movies.append(pd.DataFrame(my_favorite), ignore_index=True)

# 확인
movies.tail(10)

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller
3883,3953,About Time (2013),"Comedy, Drama, Fantasy"
3884,3954,If Only (2004),"Comedy, Drama, Fantasy"
3885,3955,Avengers: Endgame (2019),"Action, Adventure, Drama"
3886,3956,Spider-Man: Far From Home (2019),"Action, Adventure, Sci-Fi"
3887,3957,Iron Man Three (2013),"Action, Adventure, Sci-Fi"


In [12]:
# ratings의 마지막 user_id를 확인
ratings.tail()

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [13]:
# 추가할 user_id가 있는지 검사
ratings['user_id'].isin([6041]).any()

False

In [14]:
# ratings에 data를 추가한다.
my_favorites = [
    {'user_id': 6041, 'movie_id': 3953, 'count': 5},
    {'user_id': 6041, 'movie_id': 3954, 'count': 5},
    {'user_id': 6041, 'movie_id': 3955, 'count': 5},
    {'user_id': 6041, 'movie_id': 3956, 'count': 5},
    {'user_id': 6041, 'movie_id': 3957, 'count': 5},
]
my_favorites = pd.DataFrame(my_favorites)

if not ratings['user_id'].isin([6041]).any():
    ratings = ratings.append(my_favorites, ignore_index=True)

ratings.tail(10)

,user_id,movie_id,count
836473,6040,1090,3
836474,6040,1094,5
836475,6040,562,5
836476,6040,1096,4
836477,6040,1097,4
836478,6041,3953,5
836479,6041,3954,5
836480,6041,3955,5
836481,6041,3956,5
836482,6041,3957,5


### 4) CSR matrix를 직접 만들어 봅시다.

---

In [22]:
# ratings에서의 user_id, movie_id의 고유값 인덱싱
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v: k for k, v in enumerate(user_unique)}
movie_to_idx = {v: k for k, v in enumerate(movie_unique)}

print(len(user_to_idx))
print(len(movie_to_idx))

6040
3633


In [25]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다.
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다.
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):  # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다.
    ratings['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다.
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
836478,6039,3628,5
836479,6039,3629,5
836480,6039,3630,5
836481,6039,3631,5


In [46]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)),
                      shape=(num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

`ratings['count']`를 `ratings.count`로 작성해 에러가 발생했었다.  
method와 중복되는 column명은 위 방식으로 사용할 수 없나보다.

In [48]:
print(csr_data.data)
print(csr_data.indices)
print(csr_data.indptr)

[5 3 3 ... 5 5 5]
[   0    1    2 ... 3630 3631 3632]
[     0     53    169 ... 836202 836478 836483]


### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

---

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

---

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

---

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

---

### 루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

|평가문항|상세기준|
|:---|:---|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|